In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import pickle

from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

#Change this line based on your PC configurations.
path = "C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Data/"

# Change this line based on your PC configurations.
model_path = "C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Model/"

In [2]:
test_data1 = pd.read_parquet(path + "test_data1.parquet")
test_data2 = pd.read_parquet(path + "test_data2.parquet")
test_df = pd.concat([test_data1, test_data2])
del test_data1
del test_data2 

In [3]:
# Apply memory reduction for df_test since the data is way too big for memory in default settings.
for column in test_df.select_dtypes(include=['int']).columns:
    if test_df[column].max() <= np.iinfo(np.int8).max and test_df[column].min() >= np.iinfo(np.int8).min:
        test_df[column] = test_df[column].astype('int8')
    elif test_df[column].max() <= np.iinfo(np.int16).max and test_df[column].min() >= np.iinfo(np.int16).min:
        test_df[column] = test_df[column].astype('int16')
    elif test_df[column].max() <= np.iinfo(np.int32).max and test_df[column].min() >= np.iinfo(np.int32).min:
        test_df[column] = test_df[column].astype('int32')

for column in test_df.select_dtypes(include=['float']).columns:
    if test_df[column].max() <= np.finfo(np.float16).max and test_df[column].min() >= np.finfo(np.float16).min:
        test_df[column] = test_df[column].astype('float16')
    elif test_df[column].max() <= np.finfo(np.float32).max and test_df[column].min() >= np.finfo(np.float32).min:
        test_df[column] = test_df[column].astype('float32')

In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11363762 entries, 0 to 11363761
Columns: 190 entries, customer_ID to D_145
dtypes: float32(177), object(13)
memory usage: 8.6+ GB


In [5]:
test_df['S_2'] = test_df['S_2'].str.replace("-", "").str[0:6]

In [6]:
# in order to make the datasets equal, we need to eliminate these columns since we already did it in train set.
train_drop_list = ["D_87","D_88","D_108","D_111","D_110","B_39","D_73","B_42","D_134","D_135","D_136","D_137","D_138","R_9","B_29","D_106","D_132","D_49","R_26","D_76","D_66","D_42","D_142","D_53","D_82","D_50","B_17","D_105","D_56","S_9"]
test_df.drop(train_drop_list, axis = 1, inplace = True)

In [7]:
null_percentages = test_df.isnull().sum() * 100 / len(test_df)
null_percentages.sort_values(ascending = False).head(30)

D_77     49.040318
D_43     30.030777
S_27     22.002177
D_46     20.214723
D_62     16.645139
S_3      15.320323
S_7      15.320323
D_48     12.734656
D_61     10.913296
D_55      6.538090
R_27      5.838771
D_44      4.735800
D_78      4.735800
P_3       3.947452
D_91      3.875451
D_64      3.324938
D_68      3.066731
D_69      2.536766
D_83      2.536766
D_123     2.501293
D_122     2.501293
D_124     2.501293
D_119     2.501293
D_125     2.501293
D_118     2.501293
D_114     2.501293
D_117     2.501293
D_116     2.501293
D_115     2.501293
D_121     2.501293
dtype: float64

# Categorical data null analysis and conversion

In [8]:
# Find the columns that contains the categorical values.
cat_columns = list(test_df.select_dtypes('object').columns)

In [9]:
# Calculate and sort the null percentages for each column.
null_percentages = test_df[cat_columns].isnull().sum() * 100 / len(test_df)
null_percentages.sort_values(ascending = False).head(30)

D_64           3.324938
D_68           3.066731
D_114          2.501293
D_116          2.501293
D_117          2.501293
D_120          2.501293
D_126          0.725121
B_30           0.025669
B_38           0.025669
customer_ID    0.000000
S_2            0.000000
D_63           0.000000
dtype: float64

In [10]:
# Check the value frequencies for D_64 column.
test_df.D_64.value_counts()

O    5993483
U    2952811
R    2039630
Name: D_64, dtype: int64

In [11]:
# Fill the Null values as NA, since it is a deliquency (minor crime, especially that committed by young people.) field.
test_df.D_64.fillna("NA", inplace = True)
test_df.D_64 = test_df.D_64.str.replace("-1", "NA")

In [12]:
# Normalized by the meaning of value counts.
test_df.D_64.value_counts()

O     5993483
U     2952811
R     2039630
NA     377838
Name: D_64, dtype: int64

In [13]:
# Check the value frequencies for D_68 column.
test_df.D_68.value_counts()

6.0    5623121
5.0    2595202
4.0    1038014
3.0    1002553
2.0     487288
1.0     269088
Name: D_68, dtype: int64

In [14]:
# The field is categorical, however we observe some numeric values. It might be because of many things.
# We could put -1 as representative for Null values.
test_df.D_68.fillna("-1", inplace = True)

In [15]:
# Check the value frequencies for D_68 column.
test_df.D_68.value_counts()

6.0    5623121
5.0    2595202
4.0    1038014
3.0    1002553
2.0     487288
-1      348496
1.0     269088
Name: D_68, dtype: int64

In [16]:
# For the columns D_114, D_116, D_117, D_120 the null percentage is the same, which is interesting, again!
# Let's see if the null values are actually on the same rows. That would mean that those feature could be conditionally dependent.
test_df.D_114.value_counts()

1.0    6957846
0.0    4121675
Name: D_114, dtype: int64

In [17]:
test_df.D_114.fillna("-1", inplace = True)

In [18]:
test_df.D_114.value_counts()

1.0    6957846
0.0    4121675
-1      284241
Name: D_114, dtype: int64

In [19]:
test_df.D_116.value_counts()

0.0    11048854
1.0       30667
Name: D_116, dtype: int64

In [20]:
test_df.D_116.fillna("-1", inplace = True)

In [21]:
test_df.D_116.value_counts()

0.0    11048854
-1       284241
1.0       30667
Name: D_116, dtype: int64

In [22]:
test_df.D_117.value_counts()

-1.0    3097065
4.0     2430697
3.0     2303759
2.0     1244980
5.0      990268
6.0      796506
1.0      216246
Name: D_117, dtype: int64

In [23]:
# Filling with -2 because we already have -1. I reckon -1 also has a special meaning, however it is the most frequent value.
# Therefore for now we will keep it as -1 and nulls will be -2. This could be changed at other iterations.
test_df.D_117.fillna("-2", inplace = True)

In [24]:
test_df.D_117.value_counts()

-1.0    3097065
4.0     2430697
3.0     2303759
2.0     1244980
5.0      990268
6.0      796506
-2       284241
1.0      216246
Name: D_117, dtype: int64

In [25]:
test_df.D_120.value_counts()

0.0    8803450
1.0    2276071
Name: D_120, dtype: int64

In [26]:
test_df.D_120.fillna("-1", inplace = True)

In [27]:
test_df.D_120.value_counts()

0.0    8803450
1.0    2276071
-1      284241
Name: D_120, dtype: int64

In [28]:
test_df.D_126.value_counts()

1.0     8628867
0.0     2343069
-1.0     309425
Name: D_126, dtype: int64

In [29]:
test_df.D_126.fillna("-2", inplace = True)

In [30]:
test_df.D_126.value_counts()

1.0     8628867
0.0     2343069
-1.0     309425
-2        82401
Name: D_126, dtype: int64

In [31]:
test_df.B_30.value_counts()

0.0    9618034
1.0    1640895
2.0     101916
Name: B_30, dtype: int64

In [32]:
test_df.B_30.fillna("-1", inplace = True)

In [33]:
test_df.B_30.value_counts()

0.0    9618034
1.0    1640895
2.0     101916
-1        2917
Name: B_30, dtype: int64

In [34]:
test_df.B_38.value_counts()

2.0    4133804
3.0    2775020
1.0    2088714
5.0     876286
7.0     636911
4.0     494281
6.0     355829
Name: B_38, dtype: int64

In [35]:
test_df.B_38.fillna("-1", inplace = True)

In [36]:
test_df.B_38.value_counts()

2.0    4133804
3.0    2775020
1.0    2088714
5.0     876286
7.0     636911
4.0     494281
6.0     355829
-1        2917
Name: B_38, dtype: int64

In [37]:
customer_ids = test_df['customer_ID']
test_df.drop("customer_ID", axis = 1,inplace = True)

In [38]:
test_df.D_63.value_counts()

CO    8429213
CR    1830418
CL     917817
XZ     109754
XM      49717
XL      26843
Name: D_63, dtype: int64

In [39]:
cat_columns.remove("customer_ID")
cat_columns

['S_2',
 'D_63',
 'D_64',
 'D_68',
 'B_30',
 'B_38',
 'D_114',
 'D_116',
 'D_117',
 'D_120',
 'D_126']

In [40]:
# Time column is different, therefore we have to eliminate this column to avoid inconsistency.
test_df.drop('S_2', axis = 1, inplace = True)

In [41]:
# The imputation done in a very easy way. Therefore, for the next iterations we could change our approach based on more analysis.
# Now, it is time f<or the categorical --> numeric conversion.
test_df = pd.get_dummies(test_df)

In [42]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11363762 entries, 0 to 11363761
Columns: 198 entries, P_2 to D_126_1.0
dtypes: float32(148), uint8(50)
memory usage: 6.8 GB


# Numeric value imputation

In [43]:
# Find the columns that contains the categorical values.
numeric_columns = list(test_df.select_dtypes('float32').columns)

In [44]:
numeric_columns

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'B_6',
 'B_7',
 'B_8',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'B_13',
 'R_5',
 'D_58',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'P_4',
 'D_74',
 'D_75',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'S_18',
 'D_86',
 'R_17',
 'R_18',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',

In [45]:
# Let's find and impute the missing values for each numeric values now to make the data "ready".
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

test_df[numeric_columns] = pd.DataFrame(imp_mean.fit_transform(test_df[numeric_columns]), columns = numeric_columns)

In [46]:
# Test that we have no null values left, both for the numerical and categorical features. We expect max null values as "0".
test_df.isna().sum().max()

0

In [47]:
test_df.head(5)

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_117_4.0,D_117_5.0,D_117_6.0,D_120_-1,D_120_0.0,D_120_1.0,D_126_-1.0,D_126_-2,D_126_0.0,D_126_1.0
0,0.631348,0.001912,0.010727,0.814453,0.007545,0.168701,0.009972,0.002348,0.1619,0.007584,...,0,0,0,1,0,0,0,0,1,0
1,0.586914,0.005276,0.011024,0.811035,0.001817,0.241333,0.000166,0.009132,0.1619,0.006645,...,0,0,0,1,0,0,0,0,1,0
2,0.608887,0.003326,0.016388,1.004883,0.000114,0.267090,0.004196,0.004192,0.1619,0.009605,...,0,0,0,1,0,0,0,0,1,0
3,0.614746,0.009064,0.021667,0.816406,0.009720,0.188965,0.004124,0.015327,0.1619,0.007820,...,0,0,0,1,0,0,0,0,1,0
4,0.591797,0.238770,0.015930,0.810547,0.002026,0.180054,0.000731,0.011284,0.1619,0.009956,...,0,0,0,0,0,1,0,0,1,0


In [48]:
list(test_df.columns)

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'B_6',
 'B_7',
 'B_8',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'B_13',
 'R_5',
 'D_58',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'P_4',
 'D_74',
 'D_75',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'S_18',
 'D_86',
 'R_17',
 'R_18',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',

# Insert your ML Model here.

In [49]:
model_loaded = pickle.load(open(model_path + "xgb_initial.pkl", "rb"))

In [50]:
val_hat_proba = model_loaded.predict_proba(test_df)
val_hat_proba

C:\Users\Arikhan Arik\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


array([[0.5526286 , 0.44737142],
       [0.70788276, 0.29211727],
       [0.7677625 , 0.23223753],
       ...,
       [0.68995345, 0.31004658],
       [0.8114777 , 0.1885223 ],
       [0.81414163, 0.18585838]], dtype=float32)

In [51]:
val_hat_df = pd.DataFrame()
val_hat_df['customer_ID'] = customer_ids
val_hat_df['prediction'] = val_hat_proba[:, 0]


In [53]:
val_hat_df

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.447371
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.292117
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.232238
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.206726
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.203028
...,...,...
11363757,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,0.366838
11363758,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,0.310047
11363759,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,0.310047
11363760,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,0.188522


In [54]:
val_hat_df = (val_hat_df
          .groupby('customer_ID')
          .mean())

# Scale the mean P_2 by the max value and take the compliment
val_hat_df['prediction'] = 1.0 - (val_hat_df['prediction'] / val_hat_df['prediction'].max())

In [66]:
val_hat_df['customer_ID'] = list(val_hat_df.index)
val_hat_df.reset_index(drop = True, inplace = True)

,prediction,cusotmer_ID
0,0.770896,00000469ba478561f23a92a868bd366de6f6527a684c9a...
1,0.995828,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...
2,0.953986,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...
3,0.356691,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...
4,0.275506,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...
...,...,...
924616,0.957218,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...
924617,0.407328,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...
924618,0.242142,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...
924619,0.449553,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...


In [67]:
# shift column 'C' to first position
first_column = val_hat_df.pop('customer_ID')
  
# insert column using insert(position,column_name,first_column) function
val_hat_df.insert(0, 'customer_ID', first_column)

In [71]:
val_hat_df.to_csv(path + "submission2.csv", index = False)